# AutoGEN 第三课： 如何调用函数处理一些更难的任务

在大多数情况下，我们更希望通过调用一些自定义的函数来解决特定任务。那么应该如何来做呢？
1. 首先应该构建好你需要的函数
2. 将你需要调用的函数名称、功能描述、参数信息等告诉给assistant agent， 相当于让这个agent拥有了知识，它会在恰当的时候跟你的user proxy agent说： “小老弟， 完成这个任务，你可以调用这个函数”
3. 将这个函数注册到user proxy agent， 这样，当其他的assistant agent告诉它可以调用这个函数的解决任务的时候， 它就会自动调用注册到自己的函数。

可能听到这里，你可能会有一些疑惑，不要紧，我举一个例子来说明：
前文我们提到， assistant agent本身没有执行代码的能力， 它能做的就只是： 别人问他问题，他根据已有的知识（大模型能力）来回答。如果你把一个函数的功能、用法、描述告诉他，他就拥有了这个知识。
比如我们告诉assistant agent： 调用“剃成光头”函数就可以打败入侵xx星人。 
当你提问：“如何拯救地球，打败外星侵略者的时候”， assistant agent 会问你要不要试试调用“剃成光头”函数， 并把如何使用这个必杀函数的秘诀告诉你，如果你没有拒绝， 你的user proxy agent 会毫不犹豫的把你剃成秃子。 TOT。 （认真脸）

下面我们会用一个正经的例子，来向你阐述这个在代码中的用法。

## 1. 配置环境

1. 你需要安装autogent的库， 要求python版本>=3.8
2. 配置openai 或者 azure openai的api key, 并将key写入到一个名为 "OAI_CONFIG_LIST"的文件中，并将该文件放在项目目录下。或者将其配置到你的环境变量中。

OAI_CONFIG_LIST的内容结构如下：
```python
[
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]

In [3]:
%pip install pyautogen~=0.1.0 -q

Note: you may need to restart the kernel to use updated packages.


## 2.调用一个简单函数来实现功能

In [2]:
import math 
def calculate_circumference(radius: float):
    """ 计算一个圆的周长"""
    return float(radius) * 2.0 * math.pi


In [8]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4"],
    },
)

llm_config = {
    "functions": [
        {
            "name": "calculate_circumference",
            "description": "Calculate the circumference of a circle",
            "parameters": {
                "type": "object",
                "properties": {
                    "radius": {
                        "type": "string",
                        "description": "The radius of the circle"
                    }
                },
                "required": ["radius"]
            }
        }
    ],
    "config_list": config_list,
    "request_timeout": 120,
}

assistant_agent = autogen.AssistantAgent(
    name="assistant_agent",
    llm_config=llm_config
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    }
)

# 注册函数
user_proxy.register_function(
    function_map={
        "calculate_circumference": calculate_circumference,
    }
)

user_proxy.initiate_chat(
    assistant_agent,
    message="直径为10的圆，周长是多少？"
)

user_proxy (to assistant_agent):

直径为10的圆，周长是多少？

--------------------------------------------------------------------------------
assistant_agent (to user_proxy):

为了求解这个问题，我们需要知道圆的周长的计算公式是 𝑃=2𝜋𝑟，其中 𝑃 是圆的周长，𝑟 是圆的半径，𝜋 是一个常数，大约等于3.14159。在这个问题中，圆的直径是10，所以半径是5。

现在我们可以使用这个信息和我们的函数来计算圆的周长。

让我们来写一段代码来解决这个问题。
***** Suggested function Call: calculate_circumference *****
Arguments: 
{
  "radius": "5"
}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION calculate_circumference...
user_proxy (to assistant_agent):

***** Response from calling function "calculate_circumference" *****
31.41592653589793
********************************************************************

--------------------------------------------------------------------------------
assistant_agent (to user_proxy):

直径为10的圆，其周长约为31.42。

TERMINATE
